In [1]:
import datetime
import yaml

In [4]:
grade_scores = {
    0: 6,
    1: 4,
    2: 3,
    3: 2,
    4: 1
}

In [19]:
def score_paper(paper, explain=False, jackpot=0):
    # Are the corrections done
    if explain:
        print(f"Scoring paper for {paper['date']}")
    # convert and sum the scores
    score_values = [grade_scores.get(grade, 0) * 2 for grade in paper["grades"]]
    score = sum(score_values)
    if explain:
        print(f"\tGrades alone give {score} points.")
    # add 100% bonus - 6 additional points
    if all(grade==jackpot for grade in paper["grades"]):
        score += 6
        if explain:
            print(f"\tBonus! 100% - adding 6 points.  Total is {score}")
    # Now score the duration - 20 minutes is optimal.
    max_duration = min(paper['duration'], 40)
    score += int(max_duration / 10)
    if explain:
        print(f"\tAdding a time bonus. Total is {score}")

    if max_duration > 15 and max_duration < 25:
        score += 4
        if explain:
            print(f"\tClose to 20! Bonus of 4 points! Total is {score}")


    # add before time bonus
    if paper["completed_in_time"]:
        score *= 1.5
        if explain:
            print(f"\tBonus! Completed in time - 1.5 times score! total is {score}")

    return score



In [26]:
def score_child(child):
    paper_scores = [score_paper(paper) for paper in child["papers"] if paper['corrections_completed']]
    return sum(paper_scores)

In [15]:
def summarize_scores(children):
    highest_score = (None, 0)
    for child_name in children.keys():
        score = score_child(children[child_name])
        print(f"{child_name}: {score}")
        if score > highest_score[1]:
            highest_score = (child_name, score)
    print(f"Current lead goes to {highest_score[0]}")

In [2]:
start_date_str = "2021-08-24"
start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
end_date = start_date + datetime.timedelta(days=7)
start_date, end_date

(datetime.datetime(2021, 8, 24, 0, 0), datetime.datetime(2021, 8, 31, 0, 0))

In time is defined as: 
* Before 3pm on non-school days
* Before 6pm on school days

In [27]:
with open("english_data.yaml") as children_fd:
    try:
        children = yaml.safe_load(children_fd)['children']
    except yaml.parser.ParserError as e:
        print(e)


In [24]:
score_paper(children["Helena"]["papers"][0], explain=True)

Scoring paper for 2021-08-24
	Grades alone give 50 points.
	Adding a time bonus. Total is 51
	Close to 20! Bonus of 4 points! Total is 55
	Bonus! Completed in time - 1.5 times score! total is 82.5


82.5

In [25]:
score_paper(children["Jonathan"]["papers"][0], explain=True)

Scoring paper for 2021-08-24
	Grades alone give 52 points.
	Adding a time bonus. Total is 53
	Bonus! Completed in time - 1.5 times score! total is 79.5


79.5

In [8]:
score_child(children['Helena']), score_child(children['Jonathan'])

(37.5, 0)

In [28]:
summarize_scores(children)

Helena: 0
Jonathan: 79.5
Current lead goes to Jonathan
